# SWAT+

This notebook is an example of BOA running a real environmental model, SWAT+. 

In the `swat` folder you will find a SAASBO configuration file that pulls the SWAT+ possible parameters from the SWAT+ `cal_parms.cal` file to create a high dimensional optimization problem to run on a SLURM cluster as well as a highly parallelized SLURM configuration file. Both configurations are too computationally expensive to run here as an example, but the `swat` folder contains the necessary files to run the model on your own machine or cluster. Both configurations are shown below.

Here we will run a simpler example, under the `swat/config_local_test.yaml` configuration file. This configuration file is a simplified version of the `swat` configuration file, with 8 parameters, 2 constraints, and 10 trials.

In [ ]:
# This notebook uses hidden cells to import and display things so
# that it can be ran regullary to make sure the documentation
# is up to date and not broken (as opposed to a markdown file
# that would have an exmaple written once and get out of date
# as the code base changes)

# These hidden cells are only responsible for rerunning
# the documentation to ensure it is correct.
# the actual relavent part of the documentation
# are the non hidden parts

In [ ]:
import pathlib
import boa

from IPython.display import Code

boa_paper_dir = pathlib.Path().resolve().parent
r_dir = boa_paper_dir / "swat"
config_path = r_dir / "config_local_test.yaml"
run_model_script = r_dir / "run_model.R"

config_saasbo_path = r_dir / "config_auto_saasbo.yaml"
config_parr_path = r_dir / "config_parr.yaml"

## Configuration File

### SAASBO

config_auto_saasbo.yaml

In [ ]:
Code(config_saasbo_path, language="yaml")

The SAASBO configuration file tells BOA to dynamically grab the parameters from the SWAT+ `cal_parms.cal` file and create a high dimensional optimization problem to run on a SLURM cluster.

When BOA gets the parameters from the `cal_parms.cal` file, it will create a parameter for each line in the file. The `cal_parms.cal` file is a text file that contains the SWAT+ parameters that can be optimized as well as their bounds they can be. Each line in the file is a parameter that can be optimized. Optimizing every parameter blindly with their maximum bounds is obviously not the most sound setup, but it demonstrates the flexibility of BOA to handle high dimensional optimization problems.

### Parallel

config_parr.yaml

In [ ]:
Code(config_parr_path, language="yaml")

## Run Model Wrapper Script

run_model.R

In [ ]:
Code(run_model_script)

## Running our script

To run our script we just need to pass the config file to BOA's CLI

```python
python -m boa --config-file path/to/config.yaml
```

or

```python
python -m boa -c path/to/config.yaml
```

In [ ]:
output = !python -m boa -c {config_path}  # we capture ipython terminal output to python variable
o = "\n".join(ln for ln in output)  # it comes in as a list, we convert to string
o = o.replace(str(r_dir),  "[/path/to/your/dir/]")  # replace the actual dir with a stand in for privacy reasons
Code(o)